<a href="https://colab.research.google.com/github/Shiva007-RAndom/App/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.2 MB/s eta 0:00:00


In [ ]:
pip install rank_bm25

In [ ]:
import pandas as pd
df = pd.read_csv('/content/rag_sample_qas_from_kis.csv')

In [ ]:
from langchain_community.document_loaders import DataFrameLoader
doc_loader = DataFrameLoader(df, page_content_column="ki_text")
doc = doc_loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
chunks = text_splitter.split_documents(doc)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
def faiss(query):
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  vecstore = FAISS.from_documents(chunks, embeddings)
  faretriever = vecstore.as_retriever()
  faresult = vecstore.similarity_search_with_score(query,k=3)
  faresult = [(doc[0],1-doc[1]) for doc in faresult]
  return faresult

In [ ]:
from rank_bm25 import BM25Okapi
def BM25(query,chunks):
  words = [chunk.page_content.split(" ") for chunk in chunks]
  bm25 = BM25Okapi(words)
  query_tokens = query.split(" ")
  scores = bm25.get_scores(query_tokens)
  chunks_with_scores = [(chunks[i], scores[i]) for i in range(len(chunks))]
  chunks_with_scores.sort(key=lambda x: x[1], reverse=True)
  return chunks_with_scores[:3]

In [ ]:
def Normalize(chunk_scores):
  max_score = chunk_scores[0][1]
  chunk_scores = [(chunk, score/max_score) for chunk, score in chunk_scores]
  return chunk_scores

In [ ]:
def reranker(bm25results, faresults, wbm25=0.4, wfa=0.6):
  bm25results = Normalize(bm25results)
  faresults = Normalize(faresults)
  all_chunks = {doc.page_content:doc for doc,score in bm25results+faresults}
  scores  = {}
  for chunk,score in bm25results:
    scores[chunk.page_content] = wbm25*score
  for chunk,score in faresults:
    scores[chunk.page_content] = scores.get(chunk.page_content,0)+wfa*score
  finallist=[(all_chunks[page_content],scores[page_content]) for page_content in all_chunks]

  return sorted(finallist,key=lambda x:x[1],reverse=True)[:3]

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="llama3-70b-8192", openai_api_key='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',temperature=0,openai_api_base="https://api.groq.com/openai/v1")

<ipython-input-11-90cc83a0db6e>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="llama3-70b-8192", openai_api_key='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',temperature=0,openai_api_base="https://api.groq.com/openai/v1")


In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")
query = "how to set up company email on mobile"
faresult=faiss(query)
bm25results=BM25(query,chunks)
final_res=reranker(bm25results,faresult)
l=[]
for doc in final_res:
  l.append(doc[0])
result = chain.run(input_documents=l,question=query)
print(result)

To set up company email on a mobile device, follow these steps:

**Step 1: Ensure Mobile Device Management (MDM) Profile is Installed (if required)**

If your company requires MDM for mobile devices, ensure that the profile is installed on your device.

**Step 2: Set Up Email Account on Mobile Device**

1. Go to the Settings app on your mobile device.
2. Select "Mail" or "Email" (depending on your device's operating system).
3. Tap "Add Account" or "Create a new account".
4. Select "Exchange" or "as the account type.
5. Enter your company email address and password.
6. If prompted, enter the company's email server address (e.g., mail.company.com).
7. Select the desired synchronization options (e.g., sync email, contacts, calendar).

**Step 3: Configure Email Settings**

1. In the email account settings, select the "Advanced" or "Security" option.
2. Ensure that the "Use SSL/TLS" or "Use secure connection" option is enabled.
3. Set the authentication method to "Username and Password" or

In [ ]:
import pandas as pd
import numpy as np
dft = pd.read_csv('/content/query_sentence_pairs (1).csv')

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from sentence_transformers import CrossEncoder
encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
pairs = list(zip(dft["Query"], dft["Sentence"]))
scores = encoder.predict(pairs)

In [ ]:
query_embeddings = np.array(embeddings.embed_documents(dft["Query"].tolist()))
sentence_embeddings = np.array(embeddings.embed_documents(dft["Sentence"].tolist()))

In [ ]:
combined_embeddings = np.concatenate((query_embeddings, sentence_embeddings), axis=1)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

model = nn.Sequential(
    nn.Linear(768, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 1),
)

X=torch.tensor(combined_embeddings).float()
y=torch.tensor(scores).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
crite=nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epo in range(10):
  batchsize = 8
  model.train()

  for i in range(0,len(X_train),batchsize):
    xb = X_train[i:i+batchsize]
    yb = y_train[i:i+batchsize]
    optimizer.zero_grad()
    preds = model(xb).squeeze(1)
    loss = crite(preds, yb)
    loss.backward()
    optimizer.step()


from sklearn.metrics import r2_score
model.eval()
with torch.no_grad():
    predictions = model(X_train).squeeze(1).numpy()
    true_values = y_train.numpy()
    r2 = r2_score(true_values, predictions)
    print(r2)

model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze(1).numpy()
    true_values = y_test.numpy()
    r2 = r2_score(true_values, predictions)
    print(r2)

0.9572993516921997
0.8724011182785034


In [ ]:
df = pd.read_csv('/content/synthetic_knowledge_items.csv')
from langchain_community.document_loaders import DataFrameLoader
doc_loader = DataFrameLoader(df.tail(90), page_content_column="ki_text")
doc = doc_loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
chunks = text_splitter.split_documents(doc)

In [ ]:
x,y=[],[]
for chunk in chunks:
  x.append(chunk.metadata.get('ki_topic'))
  y.append(chunk.page_content)

pairs=list(zip(x,y))
scores = encoder.predict(pairs)

In [ ]:
data_queryembed = np.array(embeddings.embed_documents(x))
data_sentenceembed = np.array(embeddings.embed_documents(y))
data_combinedembed = np.concatenate((data_queryembed, data_sentenceembed), axis=1)

In [ ]:
'''model = nn.Sequential(
    nn.Linear(768, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 1),
)'''

'model = nn.Sequential(\n    nn.Linear(768, 128),\n    nn.ReLU(),\n    nn.Dropout(0.5),\n    nn.Linear(128, 64),\n    nn.ReLU(),\n    nn.Dropout(0.5),\n    nn.Linear(64, 1),\n)'

In [ ]:
X=torch.tensor(data_combinedembed).float()
y=torch.tensor(scores).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
crite=nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epo in range(20):
  batchsize = 8
  model.train()

  for i in range(0,len(X_train),batchsize):
    xb = X_train[i:i+batchsize]
    yb = y_train[i:i+batchsize]
    optimizer.zero_grad()
    preds = model(xb).squeeze(1)
    loss = crite(preds, yb)
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze(1).numpy()
    true_values = y_test.numpy()
    r2 = r2_score(true_values, predictions)
    print(r2)

In [ ]:
def nnreranker(bm25results,faresults,query):
  all_chunks = {doc.page_content:doc for doc,score in bm25results+faresults}
  order=[]
  for chunk in all_chunks:
    data_queryembed = np.array(embeddings.embed_documents([query]))
    data_sentenceembed = np.array(embeddings.embed_documents(chunk))
    combined_embeddings = np.concatenate((query_embeddings, sentence_embeddings), axis=1)
    X=torch.tensor(combined_embeddings).float()
    model.eval()
    with torch.no_grad():
      predictions = model(X)[0]
    order.append((all_chunks[chunk],predictions))
  return sorted(order,key=lambda x:x[1],reverse=True)[:3]